In [ ]:
%pip install -r requirements.txt

This script helps rail operators visualize a customer's journey using location data from BigQuery, aiding in offering personalized deals for future journeys. It fetches the device's journey data and displays it on an interactive map with visit locations and timestamps.

- Data Retrieval: Queries BigQuery for the device's location history.
- Map Visualization: Shows visited locations with markers, connected by a polyline to represent the journey.
- Popup Info: Displays device ID and visit date at each marker.

How to Use:
1) Configure BigQuery credentials.
2) Set the device_id for the customer.
3) Run the script in a notebook to visualize the journey.

Technologies:
BigQuery, Folium, Pandas

In [ ]:
import pandas as pd
from google.cloud import bigquery
import folium
from IPython.display import display

# Configure the 
PROJECT_ID = "team-plutus-iisc"
TABLE_ID = "location.visited_location_view"
client = bigquery.Client.from_service_account_json('path-to-service-account-json-key/team-plutus-iisc.json')

# Query to fetch data
QUERY = f"""
SELECT device_id, lat_visit, lon_visit, date_visit, cluster_pred
FROM `{PROJECT_ID}.{TABLE_ID}`
WHERE device_id = @device_id
ORDER BY date_visit
"""

# Set the specific device_id to query
device_id = "8f04bf7a546cc5e48d634f289439bc1d68fcd6fca19ad59ccf3b093c5cf8d8f1"
job_config = bigquery.QueryJobConfig(
    query_parameters=[
        bigquery.ScalarQueryParameter("device_id", "STRING", device_id)
    ]
)

# Execute query
query_job = client.query(QUERY, job_config=job_config)
device_data = query_job.to_dataframe()

# Ensure data exists for the device
if device_data.empty:
    print("No data found for the given device_id.")
else:
    # Initialize map centered at the first location
    start_location = [device_data.iloc[0]["lat_visit"], device_data.iloc[0]["lon_visit"]]
    journey_map = folium.Map(location=start_location, zoom_start=12)

    # Add markers for each visit with device_id in popup
    for _, row in device_data.iterrows():
        popup_text = (
            f"<b>Device ID:</b> {row['device_id']}<br>"
            f"<b>Date:</b> {row['date_visit']}<br>"
        )
        folium.Marker(
            location=[row["lat_visit"], row["lon_visit"]],
            popup=popup_text,
        ).add_to(journey_map)

    # Optionally, connect the journey with lines
    coordinates = device_data[["lat_visit", "lon_visit"]].values.tolist()
    folium.PolyLine(coordinates, color="blue", weight=2.5, opacity=0.8).add_to(journey_map)

    # Display the map in the notebook
    display(journey_map)


